In [6]:
import pandas as pd
import seaborn as sns

In [7]:
df = pd.read_csv('vaccinations.csv') 

In [8]:
countries = ["Austria", "Germany", "Portugal","Spain", "Switzerland"] # add more countries
df.loc[(df["location"].isin(countries))].loc[(df["date"]=="2021-10-25")]  # specific date?!
#df.loc[(df["country"].isin(countries))& (df["date"]==2020-10-27)]

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred
4216,Austria,AUT,2021-10-25,11442514.0,5878560.0,5609062.0,279704.0,15949.0,21939.0,126.53,65.01,62.03,3.09,2426.0,5466.0,0.060
22709,Germany,DEU,2021-10-25,111097937.0,57559637.0,55181203.0,1651090.0,107670.0,130759.0,132.42,68.60,65.77,1.97,1559.0,33209.0,0.040
48803,Portugal,PRT,2021-10-25,16222829.0,9017464.0,8865394.0,NaN,NaN,6164.0,159.55,88.69,87.19,NaN,606.0,1945.0,0.019
56871,Spain,ESP,2021-10-25,71366561.0,37988609.0,37250048.0,483728.0,22084.0,42043.0,152.67,81.27,79.69,1.03,899.0,12721.0,0.027
58392,Switzerland,CHE,2021-10-25,11030897.0,5722003.0,5483891.0,275.0,16793.0,19190.0,126.57,65.65,62.92,0.00,2202.0,5120.0,0.059


In [9]:
df_GDP = pd.read_csv('gdp.csv') 

In [11]:
df_GDP.loc[(df_GDP["Country Name"].isin(countries))]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
14,Austria,AUT,GDP (current US$),NY.GDP.MKTP.CD,6.592694e+09,7.311750e+09,7.756110e+09,8.374175e+09,9.169984e+09,9.994071e+09,...,4.311203e+11,4.094252e+11,4.300687e+11,4.419961e+11,3.818176e+11,3.955686e+11,4.162073e+11,4.548726e+11,4.450754e+11,4.309474e+11
37,Switzerland,CHE,GDP (current US$),NY.GDP.MKTP.CD,9.522747e+09,1.071271e+10,1.187998e+10,1.306364e+10,1.448056e+10,1.534674e+10,...,7.220382e+11,6.921097e+11,7.127481e+11,7.343966e+11,7.021496e+11,6.956007e+11,7.044785e+11,7.355393e+11,7.317674e+11,7.522480e+11
55,Germany,DEU,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.744409e+12,3.527345e+12,3.732743e+12,3.883920e+12,3.356236e+12,3.467498e+12,3.681733e+12,3.975347e+12,3.888327e+12,3.846414e+12
70,Spain,ESP,GDP (current US$),NY.GDP.MKTP.CD,1.207213e+10,1.383430e+10,1.613855e+10,1.907491e+10,2.134384e+10,2.475696e+10,...,1.478773e+12,1.324820e+12,1.354757e+12,1.369399e+12,1.195119e+12,1.232076e+12,1.309297e+12,1.420300e+12,1.393046e+12,1.281485e+12
194,Portugal,PRT,GDP (current US$),NY.GDP.MKTP.CD,3.193200e+09,3.417517e+09,3.668222e+09,3.905734e+09,4.235608e+09,4.687464e+09,...,2.447972e+11,2.162366e+11,2.263695e+11,2.295962e+11,1.993139e+11,2.062860e+11,2.208111e+11,2.421948e+11,2.395108e+11,2.312267e+11
